# Pipeline Scoring Example

This is a demo notebook showing how to use the `orion.analysis.analyze` function on one of the
signals from the Demo Dataset and afterwards evaluate how well the pipeline performed.

In this case, we will be processing the `S-1` signal with the `lstm_dynamic_threshold.json`
pipeline.

## 1. Load the data

In the first step, we setup the environment and load the **S-1** signal from the **Demo Dataset**,

To do so, we need to import the `orion.data.load_signal` function and call it passing
the `'S-1'` name.

In [1]:
import logging;

logging.basicConfig(level=logging.ERROR)
logging.getLogger().setLevel(level=logging.ERROR)

import warnings
warnings.simplefilter("ignore")

In [2]:
from orion.data import load_signal

train = load_signal('S-1-train')
test = load_signal('S-1-test')
train.head()

,timestamp,value
0,1222819200,-0.366359
1,1222840800,-0.394108
2,1222862400,0.403625
3,1222884000,-0.362759
4,1222905600,-0.370746


## 2. Detect anomalies using a pipeline

Once we have the data, let us try to use the LSTM pipeline to analyze it and search for anomalies.

In order to do so, we will have import the `orion.analysis.analyze` function and pass it
the loaded data and the path to the pipeline JSON that we want to use.

The output will be a ``pandas.DataFrame`` containing a table with the detected anomalies.

In [3]:
from orion.analysis import analyze

pipeline_path = '../orion/pipelines/lstm_dynamic_threshold.json'

anomalies = analyze(pipeline_path, train, test)
anomalies

Using TensorFlow backend.


Train on 2054 samples, validate on 514 samples
Epoch 1/35
2054/2054 [==============================] - 12s 6ms/step - loss: 0.1063 - mean_squared_error: 0.1063 - val_loss: 0.1073 - val_mean_squared_error: 0.1073
Epoch 2/35
2054/2054 [==============================] - 11s 5ms/step - loss: 0.1015 - mean_squared_error: 0.1015 - val_loss: 0.1080 - val_mean_squared_error: 0.1080
Epoch 3/35
2054/2054 [==============================] - 11s 5ms/step - loss: 0.1020 - mean_squared_error: 0.1020 - val_loss: 0.1057 - val_mean_squared_error: 0.1057
Epoch 4/35
2054/2054 [==============================] - 11s 5ms/step - loss: 0.1015 - mean_squared_error: 0.1015 - val_loss: 0.1053 - val_mean_squared_error: 0.1053
Epoch 5/35
2054/2054 [==============================] - 11s 5ms/step - loss: 0.1005 - mean_squared_error: 0.1005 - val_loss: 0.1059 - val_mean_squared_error: 0.1059
Epoch 6/35
2054/2054 [==============================] - 11s 5ms/step - loss: 0.1002 - mean_squared_error: 0.1002 - val_loss: 0.1

,start,end,score
0,1394323200,1399701600,0.673494


## 3. Evaluate performance

In this next step we will load some already known anomalous intervals and evaluate how
good our anomaly detection was by comparing those with our detected intervals.

For this, we will first load the known anomalies for the signal that we are using:

In [4]:
from orion.data import load_anomalies
known_anomalies = load_anomalies('S-1')
known_anomalies

,start,end
0,1392768000,1402423200


Afterwards, we pass the ground truth, the detected anomalies and the original data
to the `orion.metrics.accuracy_score` and `orion.metrics.f1_score` functions in order
to compute a score that indicates how good our anomaly detection was:

In [6]:
from orion.metrics import accuracy_score, f1_score
accuracy_score(known_anomalies, anomalies, test)

0.972987721691678

In [7]:
f1_score(known_anomalies, anomalies, test)

0.7155172413793103